In [ ]:
# !pip install fasttext
import fasttext.util
fasttext.util.download_model('ar', if_exists='ignore')  
ft = fasttext.load_model('cc.ar.300.bin')

In [ ]:
print(ft.get_dimension())
fasttext.util.reduce_model(ft, 100)
print(ft.get_dimension())


NameError: ignored

In [ ]:
print(ft.get_word_vector('الشوق شوق')) 
print(ft.get_word_vector('الشوق شوق').shape)

[ 7.11780880e-03  2.37379223e-03  1.64420679e-02  3.35068516e-02
  3.73450518e-02 -1.21406186e-02  1.35379843e-02  8.11151266e-02
  1.36960123e-03 -3.42397466e-02  1.31407660e-02 -2.70486586e-02
 -3.39174271e-03 -8.30845535e-03  1.66543126e-02  1.34029593e-02
  2.47984342e-02  1.14722185e-04 -1.35674113e-02  4.32788841e-02
 -5.46407001e-03  5.09978225e-03  8.58685281e-03 -2.83538294e-03
 -1.14843724e-02  6.17981656e-03 -3.38449888e-03  1.21432832e-02
  1.19820451e-02 -1.49548398e-02 -1.49723897e-02  9.39512253e-03
  6.13083355e-02 -2.37684492e-02 -6.22232724e-03 -1.13993492e-02
 -4.30304650e-03 -1.47225044e-03 -4.64457553e-03  4.58231894e-03
 -1.94702502e-02 -9.10794915e-05  6.67185988e-03 -6.44180877e-03
  4.20285612e-02  1.30281914e-02  1.07835103e-02  2.00059079e-02
 -1.27264801e-02 -8.70739017e-03 -1.43508622e-02  7.18011172e-04
  4.37765159e-02 -1.22452350e-02  1.45954769e-02 -2.91848090e-03
  2.38882769e-02 -2.34678071e-02  2.84602097e-03 -7.34806713e-03
  5.40037341e-02 -5.66386

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import skew
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('cleaned_train.csv')
print(df.shape)
df.head()


(6988, 3)


,text,category,stance
0,بيل غيتس تلقي لقاح كوفيد19 من غير تصوير ابر و ...,celebrity,1
1,وزير صح حد يوم تحديد هل بمؤتمروا صحفي كان ما ع...,info_news,1
2,قول رح يكونو اد مسؤولي ب لبنان ما وصل لقاح ؟ ا...,info_news,1
3,تركيا . . وزير صح فخر دين قوجة تلقي اول جرع من...,celebrity,1
4,وئام وهاب شتم دول خليجي في كل طل اعلامي تسافه ...,personal,0


In [ ]:
max_length = -1
for tweet in df['text']:
  tweet_arr = tweet.split(' ')
  max_length = max(max_length, len(tweet_arr))

print(max_length)

137


In [ ]:
embedded_text = np.zeros((len(df['text']),137*300))
print(embedded_text.shape)
for i,tweet in enumerate(df['text']):
    sentence_embedding = []
    for word in tweet.split(" "):
      sentence_embedding.extend(ft.get_word_vector(word))
    # embedded_text.
    embedded_text[i] = ft.get_word_vector(sentence_embedding)

# print(embedded_text[0])

In [ ]:
type(df['stance'])

pandas.core.series.Series

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(embedded_text, df['stance'], test_size=0.3, random_state=42,stratify=df['stance'])
# print(x_train.shape)
# print(x_test.shape)
print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
# clf = Pipeline([
#     ('vectorizer_tfidf', TfidfVectorizer(ngram_range=(1,1), max_df=0.5)),
#     ('ٌRandom Forest', RandomForestClassifier(n_estimators=100, random_state=2002))
# ])
clf = RandomForestClassifier(n_estimators=100, random_state=2002)
# clf = svm.SVC(kernel='linear', C=1.0, probability=True)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test, y_pred))